# Tratamiento SAC

In [62]:
#importo las librerías que voy a usar

import pandas as pd
from data import CWTR, CWTR_SUM, DOTACION, CUILES
from deducciones.incremental.calculo import *
import empleado as ee
from helpers.bases import tabla
import numpy as np


### Cálculo del SAC correspondiente a cada semestre

In [2]:
#filtro el df que tiene todos los conceptos y dejo solamente el del SAC

df1 = CWTR_SUM
df_sac = CWTR_SUM
df_sac = df_sac[df_sac["CCn"].eq("/S02")]

#determino una lista con los meses que corresponden a cada semestre para luego agrupar

mes_1er_Sem = [1, 2, 3, 4, 5, 6]
mes_2do_Sem = [7, 8, 9, 10, 11, 12]

# creo df con el filtro correspondiente a cada semestre

df_1erSem = df_sac[df_sac['mes'].isin(mes_1er_Sem)]
df_2doSem = df_sac[df_sac['mes'].isin(mes_2do_Sem)]

#como tengo varios sac por mes, los agrupo para que me quede uno por cada semestre

df_1erSem = df_1erSem.groupby(['legajo','CCn'])['importe'].sum().reset_index() 
df_2doSem = df_2doSem.groupby(['legajo','CCn'])['importe'].sum().reset_index() 


In [59]:
#me traigo las remuneraciones para determinar si el SAC queda gravado o exento

bases_tramos = tabla("bases_tramos")

#quito todos los valores que tengan cero ya que no debo tomarlos en el promedio 
bases_tramos_sin_ceros = bases_tramos[bases_tramos["Importe"] != 0]

#creo un df por cada semestre y agrupo segun el legajo, calculando el promedio
filtro_1erSem = bases_tramos_sin_ceros[bases_tramos_sin_ceros["mes"].isin(mes_1er_Sem)]
filtro_1erSem = filtro_1erSem.groupby(['legajo'])['Importe'].mean().reset_index() 
filtro_2doSem = bases_tramos_sin_ceros[bases_tramos_sin_ceros["mes"].isin(mes_2do_Sem)]
filtro_2doSem = filtro_2doSem.groupby(['legajo'])['Importe'].mean().reset_index()
    
#renombro las columnas para poder luego unir los df segun el legajo y determino si esta gravado o exento 
SAC_1erSem =  filtro_1erSem.rename(columns={'Importe':'Prom_SAC_1erSem'})
SAC_1erSem["Sit_1er_Sem"] = np.where(SAC_1erSem["Prom_SAC_1erSem"] >= 280792, "Gravado", "Exento")
SAC_2doSem =  filtro_2doSem.rename(columns={'Importe':'Prom_SAC_2doSem'})
SAC_2doSem["Sit_2do_Sem"] = np.where(SAC_2doSem["Prom_SAC_2doSem"] >= 330000, "Gravado", "Exento")

#hago la union de los dos df con un merge y me traigo los datos de ambas tablas (outer)
df_promedios_SAC = pd.merge(SAC_1erSem, SAC_2doSem, on="legajo", how="outer")
df_promedios_SAC.to_clipboard()

In [60]:
#guardo el archivo junto con el resto de los parquet para luego llamarlo en otro lado 
df_promedios_SAC.to_parquet('../bases-ganancias-2022/middle/df_promedios_SAC.parquet', compression='brotli')